<a href="https://colab.research.google.com/github/KamalSPadiyar/company_agents/blob/main/Finance_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


1. Build AI Agents to handle various financial tasks

2. Four agents:
  - Tool Calling - Websearch
  - For RAG based query
  - Deep Research Stock Analysis
  - Evaluation (LLM-as-a-judge) Agent

##Open-Source Tech Stack:


- Fast OS LLM Inference [Groq]: https://groq.com/
- Agentic Framework [Agno]: https://www.agno.com/
- Vector Database [PgVector]: https://pypi.org/project/pgvector/
- Embeddings [Sentence-transformers]: https://huggingface.co/sentence-transformers
- Containerization [Udocker for Colab]: https://github.com/drengskapur/docker-in-colab
- Websearch [DuckDuckGo]: https://github.com/duckduckgo

## Install Required Packages

In [ ]:
!pip install groq yfinance agno
!pip install groq duckduckgo-search newspaper4k lxml_html_clean agno
!pip install -U sqlalchemy 'psycopg[binary]' pgvector pypdf agno
!pip install udocker
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.6/857.6 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 10.9 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=a1f41cf7bb8915f1f3103e7b98896924a94c6e1c7b8a41d564150d56b7479a69
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f

In [ ]:
!python --version

Python 3.11.13


## Set Environment Variables

In [ ]:
import os
# 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
# Ask user to input the key securely (hidden input)
from getpass import getpass
os.environ['GROQ_API_KEY'] = getpass('Enter your GROQ API Key: ')

print("API key set!")


Enter your GROQ API Key: ··········
API key set!


In [ ]:
import os

api_key = os.getenv('GROQ_API_KEY')

print("Using API Key:", api_key[:4] + "..." + api_key[-4:])  # Just to verify part of it (safe)


Using API Key: gsk_...sre2


In [ ]:
# IF you want to check if the keys are indeed set

# import os
# print("Groq API key set:" if os.getenv("GROQ_API_KEY") else "Groq API key not set")

##Agent 1:
##Functional Tool Calling Capability: Web Search

In [ ]:
from textwrap import dedent
from agno.agent import Agent
from agno.models.groq import Groq
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.newspaper4k import Newspaper4kTools

In [ ]:

# Initialize the research agent with advanced journalistic capabilities
research_agent = Agent(
    model=Groq(id="llama3-70b-8192"),
    tools=[DuckDuckGoTools(), Newspaper4kTools()],
    description=dedent("""\
        You are an elite research analyst in the financial services domain.
        Your expertise encompasses:

        - Deep investigative financial research and analysis
        - fact-checking and source verification
        - Data-driven reporting and visualization
        - Expert interview synthesis
        - Trend analysis and future predictions
        - Complex topic simplification
        - Ethical practices
        - Balanced perspective presentation
        - Global context integration\
    """),
    instructions=dedent("""\
        1. Research Phase
           - Search for 5 authoritative sources on the topic
           - Prioritize recent publications and expert opinions
           - Identify key stakeholders and perspectives

        2. Analysis Phase
           - Extract and verify critical information
           - Cross-reference facts across multiple sources
           - Identify emerging patterns and trends
           - Evaluate conflicting viewpoints

        3. Writing Phase
           - Craft an attention-grabbing headline
           - Structure content in Financial Report style
           - Include relevant quotes and statistics
           - Maintain objectivity and balance
           - Explain complex concepts clearly

        4. Quality Control
           - Verify all facts and attributions
           - Ensure narrative flow and readability
           - Add context where necessary
           - Include future implications
    """),
    expected_output=dedent("""\
        # {Compelling Headline}

        ## Executive Summary
        {Concise overview of key findings and significance}

        ## Background & Context
        {Historical context and importance}
        {Current landscape overview}

        ## Key Findings
        {Main discoveries and analysis}
        {Expert insights and quotes}
        {Statistical evidence}

        ## Impact Analysis
        {Current implications}
        {Stakeholder perspectives}
        {Industry/societal effects}

        ## Future Outlook
        {Emerging trends}
        {Expert predictions}
        {Potential challenges and opportunities}

        ## Expert Insights
        {Notable quotes and analysis from industry leaders}
        {Contrasting viewpoints}

        ## Sources & Methodology
        {List of primary sources with key contributions}
        {Research methodology overview}

        ---
        Research conducted by Financial Agent
        Credit Rating Style Report
        Published: {current_date}
        Last Updated: {current_time}\
    """),
    markdown=True,
    show_tool_calls=True,
    add_datetime_to_instructions=True,
)

# User Prompt 1
research_agent.print_response("Analyze the current state and future implications \
                              of artificial intelligence in Finance",stream=True,)


Output()

In [ ]:
# User Prompt 2
research_agent.print_response("Applications of Gen AI in Financial Services",stream=True,)


Output()

In [ ]:
# User Prompt 3
research_agent.print_response("AI agentsin Financial Services",stream=True,)


Output()

## Agent 2:
## Knowledge base Query Capability: RAG (Retrieval Augmented Generation)

Since we are using OS Vector Databases, we need to use docker to launch the app and initialize the database

In [ ]:
# Copyright 2024 Drengskapur
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# @title {display-mode:"form"}
# @markdown <br/><br/><center><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/docker.svg" height="150"><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/colab.svg" height="150"></center><br/>
# @markdown <center><h1>Docker in Colab</h1></center><center>github.com/drengskapur/docker-in-colab<br/><br/><br/><b>udocker("run hello-world")</b></center><br/>
def udocker_init():
    import os
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

udocker = udocker_init()

Docker-in-Colab 1.1.0

Usage:     udocker("--help")
Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples


Now we need to initialize the PGVector. It's simpler if you are running from your local terminal but to make it work in the Colab, we need to run the below steps:

In [ ]:
# 1. First install udocker
!udocker --allow-root install

# 2. Kill existing processes and clean up
!pkill -9 -f postgres
!rm -rf /content/pgdata
!udocker --allow-root rm pgvector
!rm -f postgres.log

# 3. Create fresh directory
!mkdir -p /content/pgdata
!chmod -R 777 /content/pgdata

# 4. Pull and create container with correct image path
!udocker --allow-root pull ankane/pgvector
!udocker --allow-root create --name=pgvector ankane/pgvector

# 5. Run the container
!nohup udocker --allow-root run \
    --env="POSTGRES_DB=ai" \
    --env="POSTGRES_USER=ai" \
    --env="POSTGRES_PASSWORD=ai" \
    --env="PGDATA=/var/lib/postgresql/data/pgdata" \
    --volume="/content/pgdata:/var/lib/postgresql/data" \
    --publish="5532:5432" \
    pgvector > postgres.log 2>&1 &

# 6. Connection testing
import time
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError

def test_db_connection(max_retries=5, wait_time=10):
    db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"

    for attempt in range(max_retries):
        try:
            print(f"\nConnection attempt {attempt + 1}/{max_retries}")
            engine = create_engine(db_url)
            with engine.connect() as connection:
                result = connection.execute(text("SELECT version();"))
                version = result.fetchone()[0]
                print("✅ Successfully connected to PostgreSQL!")
                print(f"Server Version: {version}")

                # Test vector extension
                connection.execute(text("CREATE EXTENSION IF NOT EXISTS vector;"))
                print("✅ Vector extension ready!")
                return engine
        except OperationalError as e:
            print(f"Attempt {attempt + 1} failed, waiting {wait_time} seconds...")
            print("\nChecking postgres status:")
            !ps aux | grep postgres
            print("\nLatest logs:")
            !tail -n 20 postgres.log
            time.sleep(wait_time)

    return None

# 7. Apply 30 secs sleep time to wait for DB to finish set up before testing for connection
print("Waiting for database to initialize...")
time.sleep(30)

engine = test_db_connection()

if engine:
    print("\n✅ Database is ready for RAG Agent initialization!")
else:
    print("\n❌ Database connection failed. Please check the logs above.")

Error: invalid container id 
Info: downloading layer sha256:f2c967e41f72b294e2b96f25154dda38dbde3603b3be33888fb437147972f24b
Info: downloading layer sha256:c5f09b50002256f9e40253d9f3f34381edbe3ca083eb5ce77ecffc874c087995
Info: downloading layer sha256:2e3723549f1143b2c0381181709301932d6a592d8969d0827c1f0133772dfbe0
Info: downloading layer sha256:7077e54346e0cc4692391042abd0479bb02443892be7c6b1085fe7184caff826
Info: downloading layer sha256:bb153abf380255875eda2f78bb3c853520a77f3175574a91d909b5d6912c75a4
Info: downloading layer sha256:f1a157d7d7b01f004e4e758a97a38a5d10c8ce79348e5b674187a99d4f0cabda
Info: downloading layer sha256:6e662fa63f18991e2026f333e95c9670506a0c891ec82e5593bb613a627c6a96
Info: downloading layer sha256:2c35234636c95a2fed252512bb033c920753cffdd75c796da556a594845c121d
Info: downloading layer sha256:04efcdd3a2a4cbfcbdd1542bb9af0b2ff422f4e7b2cde58bfe8c61521df96056
Info: downloading layer sha256:786562b3be85b223d9577821b409d7147981e9b2c9611e0c5ec8725b1255df43
Info: downl

Define a class **Documemt QA**:
- Initialize a OS sentence-transformer embedding model
- Access the Vector DB path to load the embeddings of the PDF URL passed by user
- Initialize database, ready to run QA

In [ ]:
from typing import Union, List, Tuple
from sentence_transformers import SentenceTransformer
from agno.models.groq import Groq
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase

In [ ]:
!pip install agno sentence-transformers psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.8 MB/s eta 0:00:00


In [ ]:
# First, let's explore the agno package structure
import agno
print("Available attributes in agno:")
print([attr for attr in dir(agno) if not attr.startswith('_')])

# Try to find Agent class
import pkgutil
import agno

def find_agent_class():
    for importer, modname, ispkg in pkgutil.walk_packages(agno.__path__, agno.__name__ + "."):
        try:
            __import__(modname)
            module = eval(modname)
            if hasattr(module, 'Agent'):
                print(f"Found Agent in: {modname}")
        except:
            pass

find_agent_class()

Available attributes in agno:
['agent', 'api', 'cli', 'constants', 'document', 'embedder', 'exceptions', 'knowledge', 'media', 'memory', 'models', 'reasoning', 'run', 'storage', 'tools', 'utils', 'vectordb']
Found Agent in: agno.agent
Found Agent in: agno.agent.agent
Found Agent in: agno.app.base
Found Agent in: agno.app.fastapi.app
Found Agent in: agno.app.fastapi.async_router
Found Agent in: agno.app.fastapi.sync_router
Found Agent in: agno.app.playground.async_router
Found Agent in: agno.app.playground.operator
Found Agent in: agno.app.playground.schemas
Found Agent in: agno.app.playground.sync_router
Found Agent in: agno.app.whatsapp.async_router
Found Agent in: agno.app.whatsapp.sync_router


ERROR    `boto3` not installed. Please install it via `pip install boto3`.

ERROR    LANGDB_PROJECT_ID not set in the environment

Found Agent in: agno.eval.accuracy


WARNING  LANGDB_PROJECT_ID not set in the environment

Found Agent in: agno.team.team
Found Agent in: agno.tools.dalle
Found Agent in: agno.tools.desi_vocal
Found Agent in: agno.tools.giphy
Error importing googlemaps. Please install the package using `pip install googlemaps google-maps-places`.
Found Agent in: agno.tools.knowledge
Found Agent in: agno.tools.models.azure_openai
Found Agent in: agno.tools.models.gemini
Found Agent in: agno.tools.models.groq
Found Agent in: agno.tools.models.nebius
Found Agent in: agno.tools.models_labs
Found Agent in: agno.tools.openai
Found Agent in: agno.tools.reasoning
Found Agent in: agno.tools.thinking


ERROR    Webex tools require the `webexpythonsdk` package. Run `pip install webexpythonsdk` to install it.

ERROR    `anthropic[bedrock]` not installed. Please install it via `pip install anthropic[bedrock]`.

Found Agent in: agno.workflow.workflow


In [ ]:
from typing import Union, List, Tuple
from sentence_transformers import SentenceTransformer
from agno.models.groq import Groq
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
from agno.vectordb.pgvector import PgVector
from agno.agent import Agent

class DocumentQA:
    def __init__(self):
        # Initialize embedder
        self.embedder = self._create_embedder()
        # Initialize Groq model
        self.chat_model = Groq(id="llama3-8b-8192")
        # Database URL
        self.db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"
        self.current_knowledge_base = None
        self.agent = None

    def _create_embedder(self):
        """Create the embedding model"""
        class EmbeddingModel:
            def __init__(self):
                self.model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
                self.dimensions = 384

            def get_embedding_and_usage(self, text: Union[str, List[str]]) -> Tuple[Union[List[List[float]], List[float]], dict]:
                if isinstance(text, str):
                    embedding = self.model.encode(text)
                    embedding_list = embedding.tolist()
                    usage = {"prompt_tokens": len(text.split()), "total_tokens": len(text.split())}
                    return embedding_list, usage
                else:
                    embeddings = self.model.encode(text)
                    embedding_list = embeddings.tolist()
                    total_tokens = sum(len(t.split()) for t in text)
                    usage = {"prompt_tokens": total_tokens, "total_tokens": total_tokens}
                    return embedding_list, usage

            def get_embedding(self, text: Union[str, List[str]]) -> Union[List[float], List[List[float]]]:
                if isinstance(text, str):
                    return self.model.encode(text).tolist()
                return self.model.encode(text).tolist()

        print("✅ Embedding model(sentence-transformers/paraphrase-MiniLM-L6-v2) initialized successfully!")
        return EmbeddingModel()

    def load_pdf_url(self, url: str, table_name: str = "documents"):
        """Load a PDF from a URL"""
        try:
            # Create PDF URL knowledge base
            self.current_knowledge_base = PDFUrlKnowledgeBase(
                urls=[url],
                vector_db=PgVector(
                    table_name=table_name,
                    db_url=self.db_url,
                    embedder=self.embedder
                ),
            )

            # Initialize the Agent
            self.agent = Agent(
                knowledge=self.current_knowledge_base,
                search_knowledge=True,
                model=self.chat_model
            )

            # Load knowledge base
            print("Loading knowledge base...")
            self.current_knowledge_base.load(recreate=True)
            print("✅ Knowledge base loaded successfully!")

            # Show sample content
            self.show_sample_content()

        except Exception as e:
            print(f"❌ Error loading PDF: {e}")
            import traceback
            print(traceback.format_exc())

    def show_sample_content(self, num_samples: int = 5):
        """Show sample content from the knowledge base"""
        try:
            if not self.current_knowledge_base:
                print("No knowledge base loaded!")
                return

            docs = self.current_knowledge_base.search("")
            print("\nSample documents in knowledge base:")
            print("-" * 50)
            for i, doc in enumerate(docs[:num_samples], 1):
                print(f"\nDocument {i}:")
                if hasattr(doc, 'content'):
                    print(doc.content[:200] + "..." if len(doc.content) > 200 else doc.content)
                elif hasattr(doc, 'text'):
                    print(doc.text[:200] + "..." if len(doc.text) > 200 else doc.text)
        except Exception as e:
            print(f"Error showing samples: {e}")

    def ask(self, question: str):
        """Ask a question about the loaded document"""
        if not self.current_knowledge_base or not self.agent:
            print("Please load a document first!")
            return

        print(f"\nQ: {question}")
        try:
            # Get relevant documents
            relevant_docs = self.current_knowledge_base.search(question)
            print("\nRelevant documents found:", len(relevant_docs) if relevant_docs else 0)

            # Build context from relevant documents
            context = "\n".join([doc.content if hasattr(doc, 'content') else doc.text
                               for doc in relevant_docs])

            # Create a prompt that includes the context
            full_prompt = f"""Based on the following content:{context}
            Question: {question}
            Please provide a detailed answer based ONLY on the information provided above."""

            # Get response with context
            response = self.agent.run(full_prompt)
            print(f"\nA: {response.content}")

        except Exception as e:
            print(f"Error: {e}")
            import traceback
            print(traceback.format_exc())

# Example usage:
# qa = DocumentQA()
# qa.load_pdf_url("https://example.com/document.pdf")
# qa.ask("What is the main topic of this document?")

In [ ]:
# Example usage:
qa = DocumentQA()
qa.load_pdf_url("https://www.apple.com/environment/pdf/Apple_Environmental_Progress_Report_2024.pdf")
qa.ask("What is the main topic of this document?")

✅ Embedding model(sentence-transformers/paraphrase-MiniLM-L6-v2) initialized successfully!


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

...

✅ Knowledge base loaded successfully!


INFO Found 5 documents


Sample documents in knowledge base:
--------------------------------------------------

Document 1:
Environmental progress can and should be good for business. We underpin our climate strategy with strong business principles and innovation while harnessing the power of markets to replicate our solut...

Document 2:
Supporting communities worldwide Through our engagement efforts, we work directly with groups and individuals who are addressing environmental injustice in their communities. We evaluate each opportun...

Document 3:
 Environmental Progress Report 104Engagement and AdvocacyEnvironmental Initiatives DataIntroduction Contents Appendix

Document 4:
      2024 Environmental Progress Report 102Engagement and AdvocacyEnvironmental Initiatives DataIntroduction Contents Appendix

Document 5:
An ambitious goal for 2030: We committed to be carbon neutral for our entire carbon footprint by the end of the decade. Our journey to 2030 centers on reducing our scope 1, 2, and 3 emissi

INFO Found 5 documents


Relevant documents found: 5

A: The main topic of this document is Apple's commitment to creating products of the highest quality and their efforts to ensure the health and environmental impacts of their products. The document highlights their Full Material Disclosure (FMD) program, Chemical Safety Disclosure (CSD) program, and Regulated Substances Specification (RSS) to manage materials and chemicals used in their products, as well as their efforts to reduce potential toxicological risk and promote responsible circular economies.


In [ ]:
class DocumentQA:
    def __init__(self):
        # Initialize embedder
        self.embedder = self._create_embedder()
        # Initialize Groq model
        self.chat_model = Groq(id="llama3-8b-8192")
        # Database URL
        self.db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"
        self.current_knowledge_base = None
        self.agent = None

    def _create_embedder(self):
        """Create the embedding model"""
        class EmbeddingModel:
            def __init__(self):
                self.model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
                self.dimensions = 384

            def get_embedding_and_usage(self, text: Union[str, List[str]]) -> Tuple[Union[List[List[float]], List[float]], dict]:
                if isinstance(text, str):
                    embedding = self.model.encode(text)
                    embedding_list = embedding.tolist()
                    usage = {"prompt_tokens": len(text.split()), "total_tokens": len(text.split())}
                    return embedding_list, usage
                else:
                    embeddings = self.model.encode(text)
                    embedding_list = embeddings.tolist()
                    total_tokens = sum(len(t.split()) for t in text)
                    usage = {"prompt_tokens": total_tokens, "total_tokens": total_tokens}
                    return embedding_list, usage

            def get_embedding(self, text: Union[str, List[str]]) -> Union[List[float], List[List[float]]]:
                if isinstance(text, str):
                    return self.model.encode(text).tolist()
                return self.model.encode(text).tolist()

        print("✅ Embedding model(sentence-transformers/paraphrase-MiniLM-L6-v2) initialized successfully!")
        return EmbeddingModel()

    def load_pdf_url(self, url: str, table_name: str = "documents"):
        """Load a PDF from a URL"""
        try:
            # Create PDF URL knowledge base
            self.current_knowledge_base = PDFUrlKnowledgeBase(
                urls=[url],
                vector_db=PgVector(
                    table_name=table_name,
                    db_url=self.db_url,
                    embedder=self.embedder
                ),
            )

            # Initialize the Agent
            self.agent = Agent(
                knowledge=self.current_knowledge_base,
                search_knowledge=True,
                model=self.chat_model
            )

            # Load knowledge base
            print("Loading knowledge base...")
            self.current_knowledge_base.load(recreate=True)
            print("✅ Knowledge base loaded successfully!")

            # Show sample content
            self.show_sample_content()

        except Exception as e:
            print(f"❌ Error loading PDF: {e}")
            import traceback
            print(traceback.format_exc())

    def show_sample_content(self, num_samples: int = 5):
        """Show sample content from the knowledge base"""
        try:
            if not self.current_knowledge_base:
                print("No knowledge base loaded!")
                return

            docs = self.current_knowledge_base.search("")
            print("\nSample documents in knowledge base:")
            print("-" * 50)
            for i, doc in enumerate(docs[:num_samples], 1):
                print(f"\nDocument {i}:")
                if hasattr(doc, 'content'):
                    print(doc.content[:200] + "..." if len(doc.content) > 200 else doc.content)
                elif hasattr(doc, 'text'):
                    print(doc.text[:200] + "..." if len(doc.text) > 200 else doc.text)
        except Exception as e:
            print(f"Error showing samples: {e}")

    def ask(self, question: str):
        """Ask a question about the loaded document"""
        if not self.current_knowledge_base or not self.agent:
            print("Please load a document first!")
            return

        print(f"\nQ: {question}")
        try:
            # Get relevant documents
            relevant_docs = self.current_knowledge_base.search(question)
            print("\nRelevant documents found:", len(relevant_docs) if relevant_docs else 0)

            # Build context from relevant documents
            context = "\n".join([doc.content if hasattr(doc, 'content') else doc.text
                               for doc in relevant_docs])

            # Create a prompt that includes the context
            full_prompt = f"""Based on the following content:{context}
            Question: {question}
            Please provide a detailed answer based ONLY on the information provided above."""

            # Get response with context
            response = self.agent.run(full_prompt)
            print(f"\nA: {response.content}")

        except Exception as e:
            print(f"Error: {e}")
            import traceback
            print(traceback.format_exc())

In [ ]:
# First, make sure you have scikit-learn installed
# pip install scikit-learn

# Then run your code
RAG_qa = DocumentQA()

# Load a PDF (replace with your actual PDF URL)
RAG_qa.load_pdf_url("https://www.apple.com/environment/pdf/Apple_Environmental_Progress_Report_2024.pdf")

# Ask questions
RAG_qa.ask("What is the main topic of this document?")

✅ Embedding model(sentence-transformers/paraphrase-MiniLM-L6-v2) initialized successfully!
Loading knowledge base...


INFO Dropping collection

INFO Table 'ai.documents' dropped successfully.

INFO Creating collection

INFO Loading knowledge base

...

✅ Knowledge base loaded successfully!


INFO Found 5 documents


Sample documents in knowledge base:
--------------------------------------------------

Document 1:
Environmental progress can and should be good for business. We underpin our climate strategy with strong business principles and innovation while harnessing the power of markets to replicate our solut...

Document 2:
Supporting communities worldwide Through our engagement efforts, we work directly with groups and individuals who are addressing environmental injustice in their communities. We evaluate each opportun...

Document 3:
 Environmental Progress Report 104Engagement and AdvocacyEnvironmental Initiatives DataIntroduction Contents Appendix

Document 4:
      2024 Environmental Progress Report 102Engagement and AdvocacyEnvironmental Initiatives DataIntroduction Contents Appendix

Document 5:
An ambitious goal for 2030: We committed to be carbon neutral for our entire carbon footprint by the end of the decade. Our journey to 2030 centers on reducing our scope 1, 2, and 3 emissi

INFO Found 5 documents


Relevant documents found: 5

A: The main topic of this document is Apple's commitment to creating products of the highest quality, including their efforts to disclose detailed information about the materials used in their products, ensuring the health and environmental impacts of everything they make.


In [ ]:
RAG_qa.load_pdf_url("https://www.apple.com/environment/pdf/Apple_Environmental_Progress_Report_2024.pdf")

Loading knowledge base...


INFO Dropping collection

INFO Table 'ai.documents' dropped successfully.

INFO Creating collection

INFO Loading knowledge base

...

INFO Inserted batch of 1 documents.

INFO Added 131 documents to knowledge base

✅ Knowledge base loaded successfully!


INFO Found 5 documents


Sample documents in knowledge base:
--------------------------------------------------

Document 1:
Environmental progress can and should be good for business. We underpin our climate strategy with strong business principles and innovation while harnessing the power of markets to replicate our solut...

Document 2:
Supporting communities worldwide Through our engagement efforts, we work directly with groups and individuals who are addressing environmental injustice in their communities. We evaluate each opportun...

Document 3:
 Environmental Progress Report 104Engagement and AdvocacyEnvironmental Initiatives DataIntroduction Contents Appendix

Document 4:
      2024 Environmental Progress Report 102Engagement and AdvocacyEnvironmental Initiatives DataIntroduction Contents Appendix

Document 5:
An ambitious goal for 2030: We committed to be carbon neutral for our entire carbon footprint by the end of the decade. Our journey to 2030 centers on reducing our scope 1, 2, and 3 emissi

In [ ]:
RAG_qa.ask("Key points in this report? Give in 5 bullets")


Q: Key points in this report? Give in 5 bullets


INFO Found 5 documents

Exception ignored in: <function SyncHttpxClientWrapper.__del__ at 0x79403cf53b00>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/groq/_base_client.py", line 799, in __del__
    if self.is_closed:
       ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/httpx/_client.py", line 228, in is_closed
    return self._state == ClientState.CLOSED
           ^^^^^^^^^^^
AttributeError: 'SyncHttpxClientWrapper' object has no attribute '_state'
Exception ignored in: <function SyncHttpxClientWrapper.__del__ at 0x79403cf53b00>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/groq/_base_client.py", line 799, in __del__
    if self.is_closed:
       ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/httpx/_client.py", line 228, in is_closed
    return self._state == ClientState.CLOSED
           ^^^^^^^^^^^
AttributeError: 'SyncHttpxClientWrapper' object has no attribute '_state'



Relevant documents found: 5

A: Here are the key points in 5 bullets:

• Apple has a "Regulated Substances Specification" that sets requirements for the use of chemicals or materials in its products, accessories, manufacturing processes, and packaging, which goes beyond local regulatory requirements to protect workers' health and the environment.

• The company has a Green Chemistry Advisory Board that provides feedback on key initiatives, including updates to the RSS, and applies controls to materials that come into prolonged skin contact to minimize the potential for skin reactions.

• Apple evaluates the safety of its products and materials through chemical analyses at its Environmental Testing Lab, where chemists test materials to monitor compliance with specifications, and toxicological assessments are performed on new materials to proactively evaluate and eliminate potentially harmful substances.

• The company collects data on chemicals and materials through disclosure programs

In [ ]:
RAG_qa.ask("Executive Summary in 100 words")


Q: Executive Summary in 100 words


INFO Found 5 documents


Relevant documents found: 5

A: Here is the executive summary in 100 words:

Apple engaged Apex Companies, LLC to conduct an independent assurance of select environmental data reported in its 2023 environmental report. The scope of work included assurance of energy, renewable energy, water withdrawal, greenhouse gas emissions, apple comprehensive carbon footprint, waste quantities, and paper quantities. Apex reviewed internal and external documentary evidence, conducted site visits, and interviewed personnel to provide assurance on the accuracy, reliability, and objectivity of the reported data. The report summarizes the findings of the assurance engagement, which verified certain indicators for Apple's fiscal year 2022 reporting period.


## Agent 3:
##Stock Market analysis

1. Utilize yahoo finance to run comparative analysis using many
2. Generate a small summary report

In [ ]:
from textwrap import dedent

from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.yfinance import YFinanceTools

stock_agent = Agent(
    model=Groq(id="llama3-70b-8192"),
    tools=[
        YFinanceTools(
            stock_price=True,
            analyst_recommendations=True,
            stock_fundamentals=True,
            historical_prices=True,
            company_info=True,
            company_news=True,
        )
    ],
    instructions=dedent("""\
        You are a seasoned credit rating analyst with deep expertise in market analysis! 📊

        Follow these steps for comprehensive financial analysis:
        1. Market Overview
           - Latest stock price
           - 52-week high and low
        2. Financial Deep Dive
           - Key metrics (P/E, Market Cap, EPS)
        3. Market Context
           - Industry trends and positioning
           - Competitive analysis
           - Market sentiment indicators

        Your reporting style:
        - Begin with an executive summary
        - Use tables for data presentation
        - Include clear section headers
        - Highlight key insights with bullet points
        - Compare metrics to industry averages
        - Include technical term explanations
        - End with a forward-looking analysis

        Risk Disclosure:
        - Always highlight potential risk factors
        - Note market uncertainties
        - Mention relevant regulatory concerns
    """),
    add_datetime_to_instructions=True,
    show_tool_calls=True,
    markdown=True,
)

print("Stock Agent created. Ready to take user queries..")

Stock Agent created. Ready to take user queries..


In [ ]:

# User Query 1
stock_agent.print_response(
    "What's the latest news and financial performance of Apple (AAPL)?", stream=True)

Output()

In [ ]:
from textwrap import dedent

from agno.agent import Agent
from agno.models.groq import Groq
from agno.tools.yfinance import YFinanceTools

finance_agent = Agent(
    model=Groq(id="llama3-70b-8192"),
    tools=[
        YFinanceTools(
            stock_price=True,
            analyst_recommendations=True,
            stock_fundamentals=True,
            historical_prices=True,
            company_info=True,
            company_news=True,
            income_statements=True,
            technical_indicators=True,
        )
    ],
    instructions=dedent("""\
        You are a comprehensive financial advisor with expertise in investment analysis and portfolio management! 💰

        Follow these steps for thorough financial analysis:
        1. Company Overview
           - Business model and operations
           - Market position and competitive advantages
           - Recent news and developments

        2. Financial Performance Analysis
           - Revenue and earnings trends
           - Profitability metrics (margins, ROE, ROA)
           - Growth rates and consistency
           - Balance sheet strength
           - Cash flow analysis

        3. Valuation Assessment
           - Key valuation ratios (P/E, P/B, EV/EBITDA)
           - Price-to-growth analysis
           - Comparison to industry peers
           - Fair value estimation

        4. Technical Analysis
           - Price trends and momentum
           - Support and resistance levels
           - Technical indicators
           - Volume analysis

        5. Investment Recommendation
           - Buy/Hold/Sell recommendation
           - Target price range
           - Investment thesis
           - Time horizon considerations

        Your reporting style:
        - Start with executive summary and recommendation
        - Use tables and charts for data visualization
        - Include clear section headers with emojis
        - Highlight key insights with bullet points
        - Provide both bullish and bearish scenarios
        - Compare metrics to sector/market averages
        - Explain complex financial concepts clearly
        - End with actionable investment advice

        Risk Assessment:
        - Always identify key risk factors
        - Highlight market and company-specific risks
        - Note regulatory and competitive threats
        - Assess liquidity and volatility concerns
        - Mention macroeconomic sensitivities

        Portfolio Perspective:
        - Consider diversification benefits
        - Assess correlation with major indices
        - Evaluate sector concentration risks
        - Suggest position sizing guidelines
    """),
    add_datetime_to_instructions=True,
    show_tool_calls=True,
    markdown=True,
)

print("Finance Agent created. Ready to provide comprehensive financial analysis..")

Finance Agent created. Ready to provide comprehensive financial analysis..


In [ ]:
# User Query 2: Semiconductor market analysis
finance_agent.print_response(
    dedent("""\
    Analyze the semiconductor market performance focusing on:
    - NVIDIA (NVDA)
    - AMD (AMD)
    - Intel (INTC)
    - Taiwan Semiconductor (TSM)
    Compare their market positions, growth metrics, and future outlook in terms of AI growth."""),
    stream=True,
)



Output()

## Agent 4

## Evaluation: LLM-as-a-judge

In [ ]:
from textwrap import dedent
from agno.agent import Agent
from agno.models.groq import Groq

class RAGEvaluator:
    def __init__(self):
        self.evaluator = self._initialize_evaluator()

    def _initialize_evaluator(self):
        return Agent(
            model=Groq(id="llama-3.1-8b-instant"),  # Using different Llama model
            description=dedent("""\
                You are an expert RAG system evaluator with deep expertise in:
                - Information retrieval quality assessment
                - Response accuracy evaluation
                - Source attribution verification
                - Context relevance analysis
                - Natural language generation evaluation
            """),
            instructions=dedent("""\
                Evaluate the RAG system output based on these key metrics:

                1. Faithfulness (1-5):
                   - How accurately does the response reflect the source documents?
                   - Are there any hallucinations or incorrect statements?
                   - Does it maintain factual consistency?

                2. Context Relevance (1-5):
                   - Are the retrieved passages relevant to the query?
                   - Is important context missing?
                   - Is irrelevant information included?

                3. Answer Completeness (1-5):
                   - Does the response fully address the query?
                   - Are all key aspects covered?
                   - Is the level of detail appropriate?

                4. Source Attribution (1-5):
                   - Are sources properly cited?
                   - Is it clear which information comes from where?
                   - Can claims be traced back to sources?

                5. Response Coherence (1-5):
                   - Is the response well-structured?
                   - Does it flow logically?
                   - Is it easy to understand?

                Provide specific examples and explanations for each score.
            """),
            expected_output=dedent("""\
                # RAG Evaluation Report

                ## Overview
                Query: {query}
                Response Length: {n_chars} characters

                ## Metric Scores

                ### Faithfulness: {score}/5
                - Justification:
                - Examples:
                - Areas for Improvement:

                ### Context Relevance: {score}/5
                - Justification:
                - Examples:
                - Areas for Improvement:

                ### Answer Completeness: {score}/5
                - Justification:
                - Examples:
                - Areas for Improvement:

                ### Source Attribution: {score}/5
                - Justification:
                - Examples:
                - Areas for Improvement:

                ### Response Coherence: {score}/5
                - Justification:
                - Examples:
                - Areas for Improvement:

                ## Overall Score: {total}/25

                ## Key Recommendations
                1. {rec1}
                2. {rec2}
                3. {rec3}

                ## Summary
                {final_assessment}
            """),
            markdown=True,
        )

    def evaluate(self, query: str, response: str, context: list, stream: bool = True):
        """
        Evaluate a RAG system's response

        Args:
            query (str): Original user query
            response (str): RAG system's response
            context (list): Retrieved passages used for the response
            stream (bool): Whether to stream the evaluation output
        """
        evaluation_prompt = f"""
        Please evaluate this RAG system output:

        QUERY:
        {query}

        RETRIEVED CONTEXT:
        {' '.join(context)}

        RESPONSE:
        {response}

        Provide a detailed evaluation following the metrics and format specified.
        """

        return self.evaluator.print_response(evaluation_prompt, stream=stream)


# Initialize evaluator
evaluator = RAGEvaluator()
print("LLM-as-a Judge Evaluator initialized successfully!")

LLM-as-a Judge Evaluator initialized successfully!


In [ ]:
# Example evaluation. Rerun this to use actual financial RAG outputs

query = "What are the key features of transformer models?"
context = [
    "Transformer models use self-attention mechanisms to process input sequences.",
    "Key features include parallel processing and handling of long-range dependencies."
]
response = "Transformer models are characterized by their self-attention mechanism..."

# Run evaluation
evaluator.evaluate(query, response, context)

Output()

# Appendix

## Papers:
1. The Rise and Potential of Large Language Model Based Agents: A Survey: https://arxiv.org/pdf/2309.07864
2. Self-Reflection in LLM Agents: Effects on Problem-Solving Performance: https://arxiv.org/pdf/2405.06682v3
3. Agent Laboratory: Using LLM Agents as Research Assistants: https://arxiv.org/pdf/2501.04227v1

Free Courses:
1. By Hugging Face: https://huggingface.co/agents-course
2. By Aishwarya Naresh Reganti:
- https://github.com/aishwaryanr/awesome-generative-ai-guide/blob/main/resources/agents_101_guide.md
- https://github.com/aishwaryanr/awesome-generative-ai-guide/blob/main/resources/agents_roadmap.md